# One-machines Federated Learning
Run like an idiot all the cells, but first create a conda environment ```conda env create -f environment.yml``` (or manually and then run ```pip install -r requirements.txt```).


## Data creation

In [ ]:
import os
 
# Get the current working directory, remove last, and change the current working directory to the parent directory
current_path = os.getcwd()
parent_path = os.path.dirname(current_path) +'/data'
os.chdir(parent_path)
 
# Create CIFAR-10 dataset
print("\033[93m Create CIFAR-10\033[00m")
script_path = os.path.join(parent_path, 'cifar_creation.py')
!python $script_path
 
# Create MNIST dataset
print("\033[93m Create MNIST\033[00m")
script_path = os.path.join(parent_path, 'mnist_creation.py')
!python $script_path
 
# Split client datasets
parent_path = os.path.dirname(current_path)
os.chdir(parent_path)
print("\033[93m Split client datasets\033[00m")
script_path = os.path.join(parent_path, 'data/client_split.py')
!python $script_path --seed 1 --n_clients 5

## Training

In [2]:
# Parameters
n_clients = 5
n_attackers = 1
attack_type = "DP_flip" # Choose between: 'DP_flip', 'DP_inverted_loss', 'MP_noise', 'MP_gradient'
n_rounds = 20
data_type = "2cluster" # Choose between: '2cluster', 'random', (i.e., non-IID and IID)
dataset = "diabetes" # Choose between 'diabetes', 'breast', 'synthetic', 'mnist', 'cifar10'

In [ ]:
import subprocess
import time

# Fixed parameters for this example
model = "net"
pers = "0"
client_id = "2"

# Start server process
server_command = [
    "python", "server_FBPs.py", 
    "--rounds", str(n_rounds), 
    "--data_type", data_type, 
    "--dataset", dataset, 
    "--model", model, 
    "--pers", pers, 
    "--n_clients", str(n_clients), 
    "--n_attackers", str(n_attackers), 
    "--attack_type", attack_type
]
server_process = subprocess.Popen(server_command)
time.sleep(2)

client_processes = []
for i in range(n_clients):
    print("Start Client", i+1)
    # Start client process
    client_command = [
        "python", "client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset
    ]
    client_processes.append(subprocess.Popen(client_command))
    
    
client_malicious_processes = []
for i in range(n_attackers):
    print("Start Attacker", i+1)
    # Start client process
    client_command = [
        "python", "malicious_client.py", 
        "--id", str(i+1), 
        "--data_type", data_type, 
        "--model", model, 
        "--dataset", dataset, 
        "--attack_type", attack_type
    ]
    client_malicious_processes.append(subprocess.Popen(client_command))
    

server_process.wait()
for p in client_processes:
    p.wait()
for p in client_malicious_processes:
    p.wait()
